# 모델 생성과 학습

PyTorch를 활용하여,

1. 모델을 생성한다
2. 학습을 수행한다

## 토이 데이터셋 생성하기

In [ ]:
from sklearn.datasets import make_blobs

In [ ]:
x_train, y_train = make_blobs(
    n_samples=80,
    n_features=2,
    centers=[[0, 0], [1, 1], [1, 0], [0, 1]],
    shuffle=True,
    cluster_std=0.15
)

x_test, y_test = make_blobs(
    n_samples=20,
    n_features=2,
    centers=[[0, 0], [1, 1], [1, 0], [0, 1]],
    shuffle=False,
    cluster_std=0.15
)

In [ ]:
# 15번째 샘플까지만 x와 y 값을 출력합니다
for i, (x, y) in enumerate(zip(x_train, y_train)):
    print(x, y)
    if i + 1 == 15:
        break

In [ ]:
import numpy as np

In [ ]:
print(y_train)
print(y_test)

In [ ]:
# 레이블 0, 1 -> 0으로 매핑
# 레이블 2, 3 -> 1로 매핑

y_train[y_train < 2] = 0
y_train[y_train > 0] = 1
y_test[y_test < 2] = 0
y_test[y_test > 0] = 1

print(y_train)
print(y_test)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(x_train[:,0], x_train[:,1], c=y_train)
plt.scatter([0, 1, 1, 0], [0, 1, 0, 1], c='r', s=100, marker='*')
plt.show()

## 모델 생성하기

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(2, 1)

    def forward(self, x):
        x = self.fc(x)
        return F.sigmoid(x)

## 모델 학습하기

In [ ]:
import torch

from torch.optim import SGD

In [ ]:
# 학습 계산에 쓸 디바이스 설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# 데이터를 텐서로 변환
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)
x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(y_test)

In [ ]:
# 모델 생성 및 디바이스에 얹기
model = Net()

# 손실함수 결정
criterion = nn.BCELoss()

# 옵티마이저와 학습률 결정
optimizer = SGD(model.parameters(), lr=0.01)

In [ ]:
# 1회 학습하기
model.train() # 모델을 학습 모드로 설정
output = model(x_train) # 순전파
loss = criterion(output.squeeze(), y_train) # 로스 계산
loss.backward() # 역전파 (그래디언트 계산)
optimizer.step() # 역전파 (가중치 업데이트)
optimizer.zero_grad() # 그래디언트 0으로 초기화

In [ ]:
# 여러번 학습하기
train_losses = []

model.train()
for i in range(2000):
    output = model(x_train)
    loss = criterion(output.squeeze(), y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    train_losses.append(loss.item())

    if (i + 1) % 50 == 0:
        print(f'[{i + 1}] Loss: {loss:.4f}')

In [ ]:
# 학습 로스 시각화하기

plt.plot(train_losses)
plt.xlabel('Epochs')
plt.ylabel('BCE Loss')
plt.title('Train Loss')
plt.show()

## 모델 테스트하기

In [ ]:
model.eval()
with torch.no_grad():
    output = model(x_test)
    loss = criterion(output.squeeze(), y_test)
    correct = (output.squeeze().round() == y_test).sum().item()
accuracy = correct / len(y_test) * 100

print(f'Test Loss {loss.item():.4f}')
print(f'Test Accuracy {accuracy:.2f}')

## 모델 저장하고 불러오기

In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), './model.pt')

In [ ]:
# 모델 불러오기
model = Net()
model.load_state_dict(torch.load('./model.pt'))